In [50]:
import cv2
import mediapipe as mp
import json
import os
from glob import glob
from PIL import Image
import numpy as np

# Mediapipe Pose 모델 로드
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

selected_landmarks = [0, 2, 5, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 21, 23, 25, 26, 28, 30, 31, 32]

data = {
    "frames": []
}

s_idx, e_idx = 0, 5
json_count = 562

# 부모 폴더 (서브 폴더가 존재하는 폴더)
parent_folder = "D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123"
json_output_path = "D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-562.json"

# 서브 폴더 목록 가져오기 (최대 5개 선택)
while json_count < 571:
    subfolders = sorted([f.path for f in os.scandir(parent_folder) if f.is_dir()])[s_idx:e_idx]
    s_idx += 5
    e_idx += 5
    
    # Mediapipe Pose 모델 실행
    with mp_pose.Pose(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5, enable_segmentation=False) as pose:
        for folder_idx, subfolder in enumerate(subfolders):
            view_name = f"view{folder_idx+1}"  # view1, view2, ..., view5
    
            # 서브 폴더 내 이미지 파일 가져오기
            image_files = sorted(glob(os.path.join(subfolder, "*.jpg")))
    
            if not image_files:
                print(f"⚠️ {subfolder} 에서 이미지를 찾을 수 없습니다.")
                continue
    
            print(f"📂 Processing folder for {view_name}: {subfolder}")
    
            # 한 서브폴더 내 32개 이미지 저장
            for img_idx, img_file in enumerate(image_files[:32]):  # 32개 이미지 사용
                try:
                    # PIL을 사용하여 이미지 로드
                    image = Image.open(img_file).convert("RGB")
                    
                    # print(f"✅ 정상 로드: {img_file}")
                    image_np = np.array(image)
    
                    # Mediapipe에 입력할 수 있도록 배열 변환 (H, W, C 순서 유지)
                    results = pose.process(image_np)
    
                    # 키포인트 좌표 저장할 딕셔너리 (정규화된 값 유지)
                    keypoints = {}
    
                    if results.pose_landmarks:
                        for idx in selected_landmarks: 
                            landmark = results.pose_landmarks.landmark[idx]
                            keypoints[f"Point_{idx}"] = {
                                "x": landmark.x,  # 정규화된 값 (0~1)
                                "y": landmark.y   # 정규화된 값 (0~1)
                            }
    
                    # JSON 구조에 view 추가
                    frame_data = {
                        "pts": keypoints,
                        "active": "Yes" if results.pose_landmarks else "No",
                        "img_key": img_file
                    }
    
                    # 한 프레임 내에 5개의 view 포함
                    if len(data["frames"]) <= img_idx:
                        data["frames"].append({})  # 이미지 개수만큼 프레임 생성
    
                    data["frames"][img_idx][view_name] = frame_data
    
                except Exception as e:
                    print(f"❌ 이미지 처리 오류 ({img_file}): {e}")
                    continue
    
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    
    print(f"✅ 5개 View JSON 파일이 {json_output_path}에 저장되었습니다.")
    
    json_count += 1
    json_output_path = json_output_path.replace(str(json_count - 1), str(json_count))

# s_idx, e_idx = 0, 5
# parent_folder = "D:/Studying/gradu/013.피트니스자세/2.Validation/grad/1"
# json_output_path = "D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-561.json"

# for i in range(7):
#     subfolders = sorted([f.path for f in os.scandir(parent_folder) if f.is_dir()])[s_idx:e_idx]
    
#     # Mediapipe Pose 모델 실행
#     with mp_pose.Pose(static_image_mode=False, model_complexity=2, min_detection_confidence=0.5, enable_segmentation=False) as pose:
#         for folder_idx, subfolder in enumerate(subfolders):
#             view_name = f"view{folder_idx+1}"  # view1, view2, ..., view5
    
#             # 서브 폴더 내 이미지 파일 가져오기
#             image_files = sorted(glob(os.path.join(subfolder, "*.jpg")))
    
#             if not image_files:
#                 print(f"⚠️ {subfolder} 에서 이미지를 찾을 수 없습니다.")
#                 continue
    
#             print(f"📂 Processing folder for {view_name}: {subfolder}")
    
#             # 한 서브폴더 내 32개 이미지 저장
#             for img_idx, img_file in enumerate(image_files[:32]):  # 32개 이미지 사용
#                 try:
#                     # PIL을 사용하여 이미지 로드
#                     image = Image.open(img_file).convert("RGB")
                    
#                     # print(f"✅ 정상 로드: {img_file}")
#                     image_np = np.array(image)
    
#                     # Mediapipe에 입력할 수 있도록 배열 변환 (H, W, C 순서 유지)
#                     results = pose.process(image_np)
    
#                     # 키포인트 좌표 저장할 딕셔너리 (정규화된 값 유지)
#                     keypoints = {}
    
#                     if results.pose_landmarks:
#                         for idx in selected_landmarks:  # 19개 관절만 선택
#                             landmark = results.pose_landmarks.landmark[idx]
#                             keypoints[f"Point_{idx}"] = {
#                                 "x": landmark.x,  # 정규화된 값 (0~1)
#                                 "y": landmark.y   # 정규화된 값 (0~1)
#                             }
    
#                     # JSON 구조에 view 추가
#                     frame_data = {
#                         "pts": keypoints,
#                         "active": "Yes" if results.pose_landmarks else "No",
#                         "img_key": img_file
#                     }
    
#                     # 한 프레임 내에 5개의 view 포함
#                     if len(data["frames"]) <= img_idx:
#                         data["frames"].append({})  # 이미지 개수만큼 프레임 생성
    
#                     data["frames"][img_idx][view_name] = frame_data
    
#                 except Exception as e:
#                     print(f"❌ 이미지 처리 오류 ({img_file}): {e}")
#                     continue
    
#     with open(json_output_path, "w", encoding="utf-8") as f:
#         json.dump(data, f, indent=4, ensure_ascii=False)
    
#     print(f"✅ 5개 View JSON 파일이 {json_output_path}에 저장되었습니다.")

#     parent_folder = parent_folder.replace(f"grad/{str(i + 1)}", f"grad/{str(i + 2)}")
#     json_output_path = json_output_path.replace(f"body_v-{str(i + 1)}", f"body_v-{str(i + 2)}")


📂 Processing folder for view1: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\562-1-3-27-Z56_A
📂 Processing folder for view2: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\562-1-3-27-Z56_B
📂 Processing folder for view3: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\562-1-3-27-Z56_C
📂 Processing folder for view4: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\562-1-3-27-Z56_D
📂 Processing folder for view5: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\562-1-3-27-Z56_E
✅ 5개 View JSON 파일이 D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/body_v-1-562.json에 저장되었습니다.
📂 Processing folder for view1: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\563-1-3-27-Z56_A
📂 Processing folder for view2: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\563-1-3-27-Z56_B
📂 Processing folder for view3: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\563-1-3-27-Z56_C
📂 Processing folder for view4: D:/Studying/gradu/013.피트니스자세/2.Validation/grad/123\563-1-3-27-Z56_D
📂 Processing f

In [ ]:
import json
import os
import glob
import math
import numpy as np

DEFAULT_IMAGE_WIDTH = 1280
DEFAULT_IMAGE_HEIGHT = 960
MIN_SHOULDER_WIDTH = 20  # 최소 어깨 너비 설정

def compute_avg_shoulder_width(json_files):
    widths = []
    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            for frame in data["frames"]:
                for view in frame:
                    if "pts" in frame[view]:
                        keypoints = frame[view]["pts"]
                        left_shoulder = keypoints.get("Point_11", {"x": None})
                        right_shoulder = keypoints.get("Point_12", {"x": None})
                        if left_shoulder["x"] and right_shoulder["x"]:
                            widths.append(abs(left_shoulder["x"] - right_shoulder["x"]))
    return np.mean(widths) if widths else 1.0

def scale_keypoints_using_shoulder(data, avg_shoulder_width, image_width=DEFAULT_IMAGE_WIDTH, image_height=DEFAULT_IMAGE_HEIGHT):
    prev_x, prev_y = 0, 0 

    for frame in data["frames"]:
        for view in frame:
            if "pts" in frame[view]:  
                keypoints = frame[view]["pts"]

                left_shoulder = keypoints.get("Point_11", {"x": None, "y": None})
                right_shoulder = keypoints.get("Point_12", {"x": None, "y": None})

                if left_shoulder["x"] is None or right_shoulder["x"] is None:
                    continue  

                left_shoulder_x = left_shoulder["x"] * image_width
                right_shoulder_x = right_shoulder["x"] * image_width

                shoulder_width = abs(left_shoulder_x - right_shoulder_x) or avg_shoulder_width
                if shoulder_width < MIN_SHOULDER_WIDTH:
                    shoulder_width = MIN_SHOULDER_WIDTH

                for point in keypoints:
                    if keypoints[point]["x"] is None or keypoints[point]["y"] is None:
                        continue  

                    if math.isnan(keypoints[point]["x"]) or math.isnan(keypoints[point]["y"]):
                        keypoints[point]["x"], keypoints[point]["y"] = prev_x, prev_y  

                    prev_x, prev_y = keypoints[point]["x"], keypoints[point]["y"]

                for point in keypoints:
                    keypoints[point]["x"] = (keypoints[point]["x"] * image_width) / shoulder_width
                    keypoints[point]["y"] = (keypoints[point]["y"] * image_height) / shoulder_width
                    
    return data

input_folder = "D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터"
output_folder = "D:/Studying/gradu/013.피트니스자세/2.Validation/검증데이터/scaled/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

json_files = glob.glob(os.path.join(input_folder, "*.json"))
avg_shoulder_width = compute_avg_shoulder_width(json_files)

for json_file in json_files:
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)

        scaled_data = scale_keypoints_using_shoulder(json_data, avg_shoulder_width)

        file_name = os.path.basename(json_file)
        output_json_path = os.path.join(output_folder, f"scaled_{file_name}")

        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(scaled_data, f, indent=4, ensure_ascii=False)

        print(f"✅ 변환 완료: {output_json_path}")

    except Exception as e:
        print(f"❌ 파일 변환 오류: {json_file}, 오류 내용: {e}")

print("🎯 모든 JSON 파일의 변환이 완료되었습니다!")


In [ ]:
import json
import os
import glob
import math
import numpy as np

# ✅ 이미지 크기 (픽셀 단위)
DEFAULT_IMAGE_WIDTH = 1920  
DEFAULT_IMAGE_HEIGHT = 1080  
SHOULDER_MIN_WIDTH = 30  # 최소 어깨 너비 (픽셀)

def scale_keypoints_using_shoulder(data, image_width=DEFAULT_IMAGE_WIDTH, image_height=DEFAULT_IMAGE_HEIGHT):
    """
    Mediapipe 정규화된 좌표를 픽셀 단위로 변환 후, 어깨 너비로 다시 정규화하는 함수.
    """
    prev_x, prev_y = 0, 0  # NaN 값 발생 시 대체할 이전 값

    for frame in data["frames"]:
        for view in frame:
            if "pts" in frame[view]:  
                keypoints = frame[view]["pts"]

                # ✅ 어깨 좌표 확인
                left_shoulder = keypoints.get("Point_11", {"x": None, "y": None})
                right_shoulder = keypoints.get("Point_12", {"x": None, "y": None})

                # 좌표가 None이거나 비정상적인 값인지 검사
                if (left_shoulder["x"] is None or right_shoulder["x"] is None or
                    left_shoulder["x"] < 0 or right_shoulder["x"] < 0 or
                    left_shoulder["x"] > 1 or right_shoulder["x"] > 1):
                    print(f"⚠️ 어깨 좌표가 비정상적 (Point_11: {left_shoulder}, Point_12: {right_shoulder})")
                    continue  

                # ✅ 정규화된 값을 픽셀 단위로 변환
                left_shoulder_x = left_shoulder["x"] * image_width
                right_shoulder_x = right_shoulder["x"] * image_width

                # ✅ 어깨 너비 계산 (픽셀 단위)
                shoulder_width = abs(left_shoulder_x - right_shoulder_x)

                # ✅ 어깨 너비가 너무 작거나 0이면 보정
                if math.isnan(shoulder_width) or shoulder_width < SHOULDER_MIN_WIDTH:
                    print(f"⚠️ 어깨 너비 비정상: {shoulder_width:.2f}px → 최소값({SHOULDER_MIN_WIDTH}px) 적용")
                    shoulder_width = SHOULDER_MIN_WIDTH

                # ✅ NaN 값 확인 및 대체
                for point in keypoints:
                    if keypoints[point]["x"] is None or keypoints[point]["y"] is None:
                        continue  

                    if math.isnan(keypoints[point]["x"]) or math.isnan(keypoints[point]["y"]):
                        print(f"⚠️ NaN 값 발견! {point} 좌표를 이전 프레임 값으로 대체.")
                        keypoints[point]["x"] = prev_x  
                        keypoints[point]["y"] = prev_y  

                    prev_x, prev_y = keypoints[point]["x"], keypoints[point]["y"]

                # ✅ 모든 좌표를 픽셀 좌표로 변환 후 어깨 너비 기준으로 정규화
                for point in keypoints:
                    keypoints[point]["x"] = (keypoints[point]["x"] * image_width) / shoulder_width
                    keypoints[point]["y"] = (keypoints[point]["y"] * image_height) / shoulder_width
                    
    return data


# ✅ JSON 파일이 있는 폴더 지정
input_folder = "D:/Studying/gradu/013.피트니스자세/1.Training/gradu/"  # 원본 JSON 파일이 있는 폴더
output_folder = "D:/Studying/gradu/013.피트니스자세/1.Training/gradu/scaled/"  # 변환된 JSON 파일을 저장할 폴더

# ✅ 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# ✅ 폴더 내 모든 JSON 파일 가져오기
json_files = glob.glob(os.path.join(input_folder, "*.json"))

# ✅ 각 JSON 파일에 대해 스케일링 적용
for json_file in json_files:
    try:
        # JSON 파일 로드
        with open(json_file, "r", encoding="utf-8") as f:
            json_data = json.load(f)

        scaled_data = scale_keypoints_using_shoulder(json_data, image_width=1920, image_height=1080)

        # ✅ 변환된 파일 저장 경로 설정
        file_name = os.path.basename(json_file)
        output_json_path = os.path.join(output_folder, f"scaled_{file_name}")

        # ✅ 결과 저장
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(scaled_data, f, indent=4, ensure_ascii=False)

        print(f"✅ 변환 완료: {output_json_path}")

    except Exception as e:
        print(f"❌ 파일 변환 오류: {json_file}, 오류 내용: {e}")

print("🎯 모든 JSON 파일의 변환이 완료되었습니다!")
